In [2]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torchvision.io import read_image
import pandas as pd
import cv2 
import torch.nn as nn
import glob
import torchvision.models as models
import torchvision.transforms as transforms
import glob
from skimage import measure
from collections import Counter

#Change file directory path here
All the results will be stored in cw_data directory

In [3]:
from google.colab import drive
drive.mount('/content/drive')
data_dir = "/content/drive/MyDrive/Colab Notebooks/cs413/cw_data/DATA/"
try:
  results_dir = "/content/drive/MyDrive/Colab Notebooks/cs413/cw_data/results/"
  os.mkdir(results_dir)
except:
  pass

Mounted at /content/drive


In [4]:
# Function to read and write frames
def read_frames(filename):
    video_cap = cv2.VideoCapture(filename)

    count = 0
    frames = []
    while True:
        success, frame = video_cap.read()

        if success:
            count += 1
            
            # convert from BGR to RGB format
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            frames.append(frame)
        else:
            # print('Failed to read frame') # when last frame read
            break;


    print('Read ', count, ' frames in total')
    video_cap.release()
    
    return frames

def write_frames(frames, filename, codec='H264'):
    
    print('Writing frames to ', filename)
    fourcc = cv2.VideoWriter_fourcc(*codec) # 'MP4V' or 'H264'
    writer = cv2.VideoWriter(filename, fourcc, 24, (frames[0].shape[1],frames[0].shape[0]), True)

    for i in range(0,len(frames)):

        frame = cv2.cvtColor(frames[i], cv2.COLOR_RGB2BGR)

        writer.write(frame)

    
    writer.release()


In [5]:
def bounding_box(comps, label=1): # return bounding box of a componenet
    
    # array of image coordinates in x and y
    xx, yy = np.meshgrid(np.arange(0,comps.shape[1]), np.arange(0,comps.shape[0]))

    # mask/select by where value is given label (component)
    where_x = xx[comps==label]
    where_y = yy[comps==label]
    
    # find min and max extents of coordinates
    return np.min(where_x), np.min(where_y), np.max(where_x), np.max(where_y)

In [19]:
# return labels and bounding boxes of all the cards in a frame using pretrained model
# rest is same as Image extracter test function in Task-2
def card_labelBB(comps,im_gray,im):
  unique = sorted([i[0] for i in Counter(list(comps.ravel())).most_common(20)])# unique labels 

  min_size = 20000
  thres_size = 500000
  max_size = 1500000
  bounding_boxes = []
  component_labels = [] # list of component images

  for l in unique[1:]:
      bb = bounding_box(comps, label=l)
      # make a binary image for each component
      one_comp = np.zeros(im_gray.shape, dtype='uint8')
      one_comp[comps==l] = 1

      # measure its size
      n = np.count_nonzero(one_comp)

      # plot as image if it's big enough
      if (n>min_size) & (n< max_size):
          if n <= thres_size:

            contours, hierarchy = cv2.findContours(one_comp, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2:]
            # filter noisy detection
            for c in contours:
              if cv2.contourArea(c) > 10000:

                rect = cv2.minAreaRect(c)


                center, size, theta = rect
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                width = int(rect[1][0])
                height = int(rect[1][1])

                src_corners = box.astype("float32")

                dst_corners = np.array([[0, height-1],
                        [0, 0],
                        [width-1, 0],
                        [width-1, height-1]], dtype="float32")

                # the perspective transformation matrix
                M = cv2.getPerspectiveTransform(src_corners, dst_corners)

                # directly warp the rotated rectangle to get the straightened rectangle
                warpped_card = cv2.warpPerspective(im, M, (width, height))

                req_card = transform(warpped_card)
                req_card = req_card.unsqueeze(0)
                outputs = resnet(req_card)
                _, pred = torch.max(outputs.data, 1)
                

                #bounding_boxes.append(bb)
                component_labels.append(pred) 
                bounding_boxes.append(bb)
                #print('label ', l,'component size is ', n)
                
                # plt.imshow(card)
                # plt.show()

  return component_labels,bounding_boxes

In [20]:
# labels every 50th frame in the video using card_labelBB and return new sets of frames
def frame_labeller(frames):
  labelled_frames = []

  for image in frames[0:len(frames)+1:50]:

        im = np.asarray(image)
        im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

        # threshold at 248 given the distribution above
        threshed = np.zeros(im_gray.shape, 'int')
        threshed[(im[:,:,1] < 50) | (im[:,:,2] >= im[:,:,1]) | (im[:,:,0] >= im[:,:,1])] = 1

        # creates componenets
        comps = measure.label(threshed, background=0)

        labels,bndng_boxes = card_labelBB(comps,im_gray,im)

        for i in range(len(labels)):
          bgr = (0, 255, 0) # color of the box
          x1 = bndng_boxes[i][0]
          y1 = bndng_boxes[i][1]
          x2 = bndng_boxes[i][2]
          y2 = bndng_boxes[i][3]
          label = labels[i].item()
          cv2.rectangle(im,(x1, y1), (x2, y2),(1, 1, 1), 3)
          label_font = cv2.FONT_HERSHEY_SIMPLEX
          cv2.putText(im, label_mapping[label], (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)
        labelled_frames.append(im)
          
        # plt.figure(figsize = (10,10))
        # plt.imshow(im)
        # plt.show()
  return labelled_frames


Model and Label mapping Import

In [22]:
# trasnform the image to make prediction on the extracted cards from a frame
transform=transforms.Compose([transforms.ToPILImage(),
                                  #transforms.RandomResizedCrop(224), 
                                  transforms.Resize((224,224)), 
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# Model Import
resnet = models.resnet50(pretrained=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

in_features = resnet.fc.in_features
resnet.fc = nn.Linear(in_features, len(glob.glob( results_dir + "train/*")))
# importing pretrained model's state dictionary
resnet.load_state_dict(torch.load(f"{results_dir}best_resnet50_lr01_sgd.pt",map_location=torch.device('cpu')))
resnet.eval()

# Import Label mapping
annot = [i.split("/")[-1] for i in glob.glob(results_dir + "train/*")]
label_mapping = {i:idx for idx,i in enumerate(annot)}
label_mapping = dict((v,k) for k,v in label_mapping.items()) # label the original ids

cpu


#Video Import and labelling

In [23]:
# Importing the videos
frames1 = read_frames( data_dir + 'video-001.MOV')
frames2 = read_frames( data_dir + 'video-002.MOV')
# labelling the framse
# Takes around 5 mins to run
labelled_frames1 = frame_labeller(frames1)
labelled_frames2 = frame_labeller(frames2)

Read  215  frames in total
Read  522  frames in total


In [11]:
# writing the video frames
# videos won't be seen on colab, you can download and run on your system to see the outputs.
# filename1 = 'lebld_vid-001.mp4'
# filename2 = 'lebld_vid-002.mp4'
# write_frames(labelled_frames1, results_dir + filename1, codec='MP4V') # use codec=MP4V on Google Colab
# write_frames(labelled_frames2, results_dir + filename2, codec='MP4V') # use codec=MP4V on Google Colab

Writing frames to  /content/drive/MyDrive/Colab Notebooks/cs413/cw_data/results/lebld_vid-001.mp4
Writing frames to  /content/drive/MyDrive/Colab Notebooks/cs413/cw_data/results/lebld_vid-002.mp4


In [38]:
for i in labelled_frames2:
  plt.figure(figsize = (10,10))
  plt.imshow(i)
  plt.show()

Output hidden; open in https://colab.research.google.com to view.